<a href="https://colab.research.google.com/github/prasanna2019/Apache-beam-bus-travel/blob/main/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 20.7 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1

In [3]:
import apache_beam as beam
import requests
from datetime import datetime

In [4]:
class CheckZero(beam.DoFn):
  def process(self, element):
    new_val= True if(float(element[3])> 1000) else False
    yield element+[new_val]


In [5]:
def convert_data(element):
  def convert_date(date_str):
    try:
      return datetime.strptime(date_str, "%d-%m-%Y").date()
    except ValueError:
      return None

  def convert_int(int_str):
    try:
      return int(int_str)
    except ValueError:
      return None

  element[0]= convert_int(element[0])
  element[1]= convert_date(element[1])
  return element




Download and save data file from remote location

In [6]:
url = 'https://raw.githubusercontent.com/prasanna2019/pandasEDA/refs/heads/main/TFL%20Bus%20Safety.csv'
response = requests.get(url)

# Save the content to a file
file_path = '/content/TFL_Bus_Safety.csv'
with open(file_path, 'wb') as f:
    f.write(response.content)

Group Incidents By Year

In [12]:
# Step 2: Use Apache Beam to read the downloaded CSV file
with beam.Pipeline() as pipeline:
    data=(
        pipeline
        # Read the CSV file from the local path in Colab
        | 'Read from CSV' >> beam.io.ReadFromText(file_path, skip_header_lines=1)
        # Split the CSV rows into columns
        | 'Split Columns' >> beam.Map(lambda x: x.split(","))
        | 'Convert Data' >> beam.Map(lambda x: convert_data(x))
        | 'Group' >> beam.Map(lambda x: (x[0],1))
        | 'Sum' >> beam.CombinePerKey(sum)

    )
    data | 'Print Data' >> beam.Map(print)

(2015, 5715)
(2016, 6093)
(2017, 6573)
(2018, 4777)


Incidents by Borough

In [23]:
with beam.Pipeline() as pipeline:
  borough_counts = (
    pipeline |
    'Read from CSV' >> beam.io.ReadFromText(file_path, skip_header_lines=1)
    |beam.Map(lambda x: x.split(","))
 #   |"Count Total Incidents" >> beam.CombineGlobally(beam.combiners.CountCombineFn())
     | "Extract Borough Key" >> beam.Map(lambda record: (record[6], 1))
     | "Count Incidents Per Borough" >> beam.CombinePerKey(sum)
     | beam.Map(print)
  )




('Southwark', 1107)
('Islington', 769)
('Havering', 652)
('None London Borough', 561)
('Westminster', 1571)
('City of London', 243)
('Hammersmith & Fulham', 632)
('Kensington & Chelsea', 334)
('Brent', 887)
('Camden', 900)
('Hounslow', 642)
('Richmond Upon Thames', 403)
('Enfield', 584)
('Waltham Forest', 526)
('Wandsworth', 551)
('Hackney', 796)
('Barnet', 922)
('Lambeth', 1107)
('Lewisham', 1006)
('Croydon', 1032)
('Bexley', 414)
('Kingston upon Thames', 451)
('Merton', 309)
('Bromley', 673)
('Haringey', 831)
('Ealing', 863)
('Hillingdon', 517)
('Newham', 634)
('Harrow', 384)
('Sutton', 193)
('Greenwich', 832)
('Redbridge', 470)
('Tower Hamlets', 435)
('Barking & Dagenham', 374)
('Not specified', 553)
